***GENERATED CODE FOR severityclassification PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ID', 'transformation_label': 'String Indexer'}], 'feature': 'ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'A-3652164', 'max': 'A-5458016', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ID'}, {'feature_label': 'ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ID')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'Start_Date', 'transformation_label': 'Extract Date'}], 'feature': 'Start_Date', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'Start_Date'}, {'feature_label': 'Start_Date', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('Start_Date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Street', 'transformation_label': 'String Indexer'}], 'feature': 'Street', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': ' 12th Street Ext', 'max': 'Woodall Rodgers Fwy S', 'missing': '0', 'distinct': '421'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Street'}, {'feature_label': 'Street', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Street')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'City', 'transformation_label': 'String Indexer'}], 'feature': 'City', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Agoura Hills', 'max': 'Zebulon', 'missing': '0', 'distinct': '353'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'City'}, {'feature_label': 'City', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('City')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'County', 'transformation_label': 'String Indexer'}], 'feature': 'County', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Aiken', 'max': 'Yolo', 'missing': '0', 'distinct': '226'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'County'}, {'feature_label': 'County', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('County')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'State', 'transformation_label': 'String Indexer'}], 'feature': 'State', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AL', 'max': 'WY', 'missing': '0', 'distinct': '38'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'State'}, {'feature_label': 'State', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('State')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Country', 'transformation_label': 'String Indexer'}], 'feature': 'Country', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'US', 'max': 'US', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Country'}, {'feature_label': 'Country', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Country')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Wind_Direction', 'transformation_label': 'String Indexer'}], 'feature': 'Wind_Direction', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '483', 'mean': '', 'stddev': '', 'min': 'CALM', 'max': 'WSW', 'missing': '17', 'distinct': '18'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Wind_Direction'}, {'feature_label': 'Wind_Direction', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Wind_Direction')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Weather_Condition', 'transformation_label': 'String Indexer'}], 'feature': 'Weather_Condition', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '483', 'mean': '', 'stddev': '', 'min': 'Cloudy', 'max': 'Wintry Mix', 'missing': '17', 'distinct': '27'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Weather_Condition'}, {'feature_label': 'Weather_Condition', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Weather_Condition')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Sunrise_Sunset', 'transformation_label': 'String Indexer'}], 'feature': 'Sunrise_Sunset', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '498', 'mean': '', 'stddev': '', 'min': 'Day', 'max': 'Night', 'missing': '2', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Sunrise_Sunset'}, {'feature_label': 'Sunrise_Sunset', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Sunrise_Sunset')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Start_Time', 'transformation_label': 'String Indexer'}], 'feature': 'Start_Time', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '0:01:00', 'max': '9:55:00', 'missing': '0', 'distinct': '444'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Start_Time'}, {'feature_label': 'Start_Time', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Start_Time')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run severityclassificationHooks.ipynb
try:
	#sourcePreExecutionHook()

	usaccidentsdata = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/US_Accidents_Data.csv', 'filename': 'US_Accidents_Data.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(usaccidentsdata)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run severityclassificationHooks.ipynb
try:
	#transformationPreExecutionHook()

	severityclassificationautofe = TransformationMain.run(usaccidentsdata,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "ID", "transformation_label": "String Indexer"}], "feature": "ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A-3652164", "max": "A-5458016", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Severity", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.06", "stddev": "0.34", "min": "2", "max": "4", "missing": "0"}, "updatedLabel": "Severity"}, {"transformationsData": [{"feature_label": "Start_Date", "transformation_label": "Extract Date"}], "feature": "Start_Date", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "Start_Date"}, {"transformationsData": [{"feature_label": "Street", "transformation_label": "String Indexer"}], "feature": "Street", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": " 12th Street Ext", "max": "Woodall Rodgers Fwy S", "missing": "0", "distinct": "421"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Street"}, {"transformationsData": [{"feature_label": "City", "transformation_label": "String Indexer"}], "feature": "City", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Agoura Hills", "max": "Zebulon", "missing": "0", "distinct": "353"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "City"}, {"transformationsData": [{"feature_label": "County", "transformation_label": "String Indexer"}], "feature": "County", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Aiken", "max": "Yolo", "missing": "0", "distinct": "226"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "County"}, {"transformationsData": [{"feature_label": "State", "transformation_label": "String Indexer"}], "feature": "State", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AL", "max": "WY", "missing": "0", "distinct": "38"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "State"}, {"transformationsData": [{"feature_label": "Country", "transformation_label": "String Indexer"}], "feature": "Country", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "US", "max": "US", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Country"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Temperature(F)", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "482", "mean": "47.92", "stddev": "15.57", "min": "-5.0", "max": "85.0", "missing": "18"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Temperature(F)"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Wind_Chill(F)", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "482", "mean": "45.3", "stddev": "18.38", "min": "-16.0", "max": "85.0", "missing": "18"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Wind_Chill(F)"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Humidity(%)", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "480", "mean": "69.65", "stddev": "21.68", "min": "11.0", "max": "100.0", "missing": "20"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Humidity(%)"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Pressure(in)", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "483", "mean": "29.18", "stddev": "1.43", "min": "20.47", "max": "30.57", "missing": "17"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Pressure(in)"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Visibility(mi)", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "482", "mean": "8.44", "stddev": "2.99", "min": "0.0", "max": "13.0", "missing": "18"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Visibility(mi)"}, {"transformationsData": [{"feature_label": "Wind_Direction", "transformation_label": "String Indexer"}], "feature": "Wind_Direction", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "483", "mean": "", "stddev": "", "min": "CALM", "max": "WSW", "missing": "17", "distinct": "18"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Wind_Direction"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Wind_Speed(mph)", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "483", "mean": "7.89", "stddev": "5.61", "min": "0.0", "max": "26.0", "missing": "17"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Wind_Speed(mph)"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Precipitation(in)", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "472", "mean": "0.01", "stddev": "0.03", "min": "0.0", "max": "0.25", "missing": "28"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Precipitation(in)"}, {"transformationsData": [{"feature_label": "Weather_Condition", "transformation_label": "String Indexer"}], "feature": "Weather_Condition", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "483", "mean": "", "stddev": "", "min": "Cloudy", "max": "Wintry Mix", "missing": "17", "distinct": "27"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Weather_Condition"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "No_Exit", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "No_Exit"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Traffic_Calming", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.04", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Traffic_Calming"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Traffic_Signal", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.08", "stddev": "0.28", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Traffic_Signal"}, {"transformationsData": [{"feature_label": "Sunrise_Sunset", "transformation_label": "String Indexer"}], "feature": "Sunrise_Sunset", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "498", "mean": "", "stddev": "", "min": "Day", "max": "Night", "missing": "2", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Sunrise_Sunset"}, {"transformationsData": [{"feature_label": "Start_Time", "transformation_label": "String Indexer"}], "feature": "Start_Time", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "0:01:00", "max": "9:55:00", "missing": "0", "distinct": "444"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Start_Time"}]}))

	#transformationPostExecutionHook(severityclassificationautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run severityclassificationHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(severityclassificationautofe, ["Severity", "Temperature(F)", "Wind_Chill(F)", "Humidity(%)", "Pressure(in)", "Visibility(mi)", "Wind_Speed(mph)", "Precipitation(in)", "No_Exit", "Traffic_Calming", "Traffic_Signal", "ID_stringindexer", "Start_Date_dayofmonth", "Start_Date_month", "Start_Date_year", "Street_stringindexer", "City_stringindexer", "County_stringindexer", "State_stringindexer", "Country_stringindexer", "Wind_Direction_stringindexer", "Weather_Condition_stringindexer", "Start_Time_stringindexer"], "Sunrise_Sunset_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

